# KostraRegnskap og regionsaggregering

### Laster inn nødvendige pakker

In [ ]:
import pandas as pd
import numpy as np
exec(open('../Funksjoner m.m./funksjoner.py').read())

# Produserer inputfilene til KostraRegnskap
Før dette gjøres må følgende filer være kjørt:
1. API Klass 
    - Denne henter lister over følgende fra Klass: arter, fuksjoner, kommuner, fylkeskommuner, kommunehierarki, fylkeskommunehierarki og bydeler
1. Import fra Driller
    - Denne henter følgende skjemaer fra driller: 0A, 0C, 0I, 0K, 0M og 0P

## Kasse

In [ ]:
%run ./auto_bevilgning/'Kasse'.ipynb

<br>
<br>

### Kontroller utført i kasseregnskapet
**1. Kontrollerer om det er fjernet rader med null i de innleste regnskapsfilene**

In [ ]:
kontroll_merge(ac,ac_null)

<br>

**2. Kontrollerer om det er fjernet rader med ugyldige arter**


In [ ]:
kontroll_merge(forts_ac, ac_artsfilter)

<br>

**3. Kontrollerer om det er fjernet rader med ugyldige funksjoner**

In [ ]:
kontroll_merge(ac_artsfilter,ac_funksjonsfilter)

<br>

**4. Lister ut manglende kommuner**

In [ ]:
ac_manglende

## Konsolidert

In [ ]:
%run ./auto_bevilgning/'Konsolidert'.ipynb

### Kontroller utført i det konsoliderte regnskapet
**1. Kontrollerer om det er fjernet rader med null i de innleste regnskapsfilene**

In [ ]:
kontroll_merge(mp, mp_null)

In [ ]:
mp_null

In [ ]:
mp

In [ ]:
%run ./auto_bevilgning/'Saerbedriftsregnskapene'.ipynb

## Henter hierarkifilene
Disse filene er tidligere lagret på felles område på Linux. 

In [ ]:
artshierarki = pd.read_parquet('../../../Testdata/arbeidsdata/KostraRegnskap/artshierarki_'+aar+'.parquet')
funksjonshierarki = pd.read_parquet('../../../Testdata/arbeidsdata/KostraRegnskap/funksjonshierarki_'+aar+'.parquet')
formler = pd.read_parquet('../../../Testdata/arbeidsdata/KostraRegnskap/formler_'+aar+'.parquet')
stjerne = pd.read_parquet('../../../Testdata/arbeidsdata/KostraRegnskap/stjernetabell_'+aar+'.parquet')
nettinger_kasse = pd.read_parquet('../../../Testdata/arbeidsdata/KostraRegnskap/nettinger_kasse_'+aar+'.parquet')
nettinger_saer = pd.read_parquet('../../../Testdata/arbeidsdata/KostraRegnskap/nettinger_saer_'+aar+'.parquet')
kommunehierarki = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/kommunehierarki.parquet")
fylkeskommunehierarki = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/fylkeskommunehierarki.parquet")
bydelshierarki = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/bydelshierarki.parquet")

## KostraRegnskap
- Funksjonen er i R. Vi må derfor endre språk og laste over alle objekter som skal benyttes. 

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%R library(Kostra)
%R library(tidyverse)

In [ ]:
%R -i a_c_basis
%R -i kostra0i_0k_fordelt
%R -i m_p_basis
%R -i artshierarki
%R -i funksjonshierarki
%R -i formler
%R -i stjerne
%R -i nettinger_kasse
%R -i nettinger_saer
%R -i kommunehierarki
%R -i fylkeskommunehierarki
%R -i bydelshierarki
#%R -i ac_manglende

In [ ]:
%R a_c_basis$kontoklasse <- CharacterReCode(NaToZero(a_c_basis$kontoklasse), c("0", "1"), c("I", "D"))
%R kostra0i_0k_fordelt$kontoklasse <- CharacterReCode(NaToZero(kostra0i_0k_fordelt$kontoklasse), c("3", "4", "0"), c("D", "I", "I"))
%R m_p_basis$kontoklasse <- CharacterReCode(NaToZero(m_p_basis$kontoklasse), c("3", "4", "0"), c("D", "I", "I"))
%R m_p_basis$regnskapsomfang <- gsub('C','B',m_p_basis$regnskapsomfang)
%R stjerne[is.na(stjerne)] <- ""

In [ ]:
%%R resultat_konsern <- KostraRegnskap(data=a_c_basis, funksjonshierarki, artshierarki, kostra0i_0k_fordelt, nettinger_saer, nettinger_kasse, stjerne, formler,
                             regnskapsomfang="A")

## Konsern
### Aggregerer regnskapet
- Kjører funksjon HierarchyCompute fra kostra biblioteket. 
- Funksjonen aggregerer regnskapet til følgende (så sant regionsfierarkiet er korrekt):
    - EAK (Landet - kommuner)
    - EAKUO (Landet uten Oslo - kommuner)
    - EKAXX (Fylker)
    - EKG (kostragrupper)
    - EAFK (Landet - fylkeskommuner)
    - EAFKUO (Landet uten Oslo - fylkeskommuner)
    - EAFKXX (Landsdeler)
    - EAB (Bydeler)  
<br>
<br>
- Vi lager kun aggregerte størrelser på konserntall. Filtrerer derfor bort kasse før aggregeringen. 

In [ ]:
%R regionshierarki <- rbind(kommunehierarki, fylkeskommunehierarki, bydelshierarki)

In [ ]:
#%R konsern <- filter(resultat, regnskapsomfang == "A")

In [ ]:
%R aggregert <- KostraRegnskapRegionAggregering(data=resultat_konsern, hierarki=regionshierarki)

In [ ]:
%R -o resultat_konsern
%R -o aggregert

In [ ]:
aggregert['periode'] = '2021'

In [ ]:
konsern_aggregert = pd.concat([resultat_konsern, aggregert])

In [ ]:
#%R aggregert$periode <- '2021'

In [ ]:
#%R konsern_aggregert <- rbind(resultat_konsern, aggregert)

### Fjerner estimerte kommuner
- Før filen kan lagres til mappe må estimerte kommuner fjernes. Vi publiserer ikke tall for estimerte kommuner.

In [ ]:
bev_konsern = konsern_aggregert[~konsern_aggregert['region'].isin(ac_manglende)]

In [ ]:
#%%R bev_konsern <- konsern_aggregert[!konsern_aggregert$region %in%
#                                     ac_manglende$region,]

## Kasse
- Filterer ut en fil for kassetegnskapet og fjerner estimerte kommuner

In [ ]:
%%R resultat_kasse <- KostraRegnskap(data=a_c_basis, funksjonshierarki=funksjonshierarki,artshierarki=artshierarki, data_saer=NULL,
                                           artshierarki_nettinger=NULL,artshierarki_nettinger_kasse=NULL,stjernetabell=stjerne,formler=formler,
                                           regnskapsomfang="B")

In [ ]:
#%R bev_kasse <- filter(resultat_kasse, regnskapsomfang == "B")

In [ ]:
%R -o resultat_kasse

In [ ]:
bev_kasse = resultat_kasse[~resultat_kasse['region'].isin(ac_manglende)]

In [ ]:
#%%R bev_kasse <- bev_kasse[!bev_kasse$region %in%
#                           ac_manglende$region,]

## Konsolidert
- KostraRegnskaps-funksjonen er ikke laget for konsolidert. 
- Konsolidert skal behandles likt som kasse. Mekker derfor til ved å kalle sette regnskapsomfang B på konsolidert og kjøre KostraRegnskap for kun B. 
- Etter kjøringen endres regnskapsomfang tilbake til C. 

In [ ]:
%%R resultat_konsolidert <- KostraRegnskap(data=m_p_basis, funksjonshierarki=funksjonshierarki,artshierarki=artshierarki, data_saer=NULL,
                                           artshierarki_nettinger=NULL,artshierarki_nettinger_kasse=NULL,stjernetabell=stjerne,formler=formler,
                                           regnskapsomfang="B")

In [ ]:
%R -o resultat_konsolidert

In [ ]:
resultat_konsolidert['regnskapsomfang'] = resultat_konsolidert['regnskapsomfang'].str.replace('B', 'C')

In [ ]:
#%R resultat_konsolidert$regnskapsomfang <- gsub('B','C',resultat_konsolidert$regnskapsomfang)

## Samler kasse og konsolidert

In [ ]:
bev_kasse_konsolidert = pd.concat([bev_kasse, resultat_konsolidert])

In [ ]:
#%R bev_kasse_konsolidert <- rbind(bev_kasse, resultat_konsolidert)

### Skriver tilbake til Python

In [ ]:
#%R -o bev_konsern
#%R -o bev_kasse_konsolidert

## Lage filer:
Konsern uten bydel  
Konsern med bydel  
kasse_konsolidert uten bydel  

In [ ]:
bydel = bydelshierarki.rename(columns={'from':'region'})
bydelsliste = bydel.region.unique()

In [ ]:
bev_konsern_ubydel = bev_konsern[(~bev_konsern['region'].isin(bydelsliste)) &
                            (bev_konsern['region'] != 'EAB')]

In [ ]:
bev_kasse_konsolidert_ubydel = bev_kasse_konsolidert[(~bev_kasse_konsolidert['region'].isin(bydelsliste)) &
                                             (bev_kasse_konsolidert['region'] != 'EAB')]